In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
#import matplotlib as plt
import matplotlib.pyplot as plt
import sklearn.preprocessing as sp
%matplotlib inline
sns.set_style('whitegrid')
sns.set_context("poster")
sns.set_context("talk", 1.0, {"lines.linewidth": 3})

In [3]:
#パターン17のデータの取得
df19_1 = pd.read_csv("./取得データセット/受信機1/nkmr1-dataset19.csv")
df19_2 = pd.read_csv("./取得データセット/受信機2/nkmr2-dataset19.csv")
df19_3 = pd.read_csv("./取得データセット/受信機3/nkmr3-dataset19.csv")
df19_4 = pd.read_csv("./取得データセット/受信機4/nkmr4-dataset19.csv")

#それぞれのカラムの指定
df19_1.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp","x","y"]
df19_2.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp","x","y"]
df19_3.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp","x","y"]
df19_4.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp","x","y"]


df19 = df19_1.append(df19_2).append(df19_3).append(df19_4)

#同期ビーコンのid
set_uuid = "1000000007ae1001b000001c4d8dffff"

#送信機のid
id1 = "10000000aaaa41441111110000000000"
id2 = "20000000aaaa41441111110000000000"
id3 = "30000000aaaa41441111110000000000"
id4 = "40000000aaaa41441111110000000000"
id5 = "50000000aaaa41441111110000000000"
id6 = "60000000aaaa41441111110000000000"
id7 = "70000000aaaa41441111110000000000"
id8 = "80000000aaaa41441111110000000000" 
id9 = "90000000aaaa41441111110000000000"

#locateを作成
df19["locate"] = 0
df19.locate = df19.locate.where(df19.UUIDs != id1,1)
df19.locate = df19.locate.where(df19.UUIDs != id2,2)
df19.locate = df19.locate.where(df19.UUIDs != id3,3)
df19.locate = df19.locate.where(df19.UUIDs != id4,4)
df19.locate = df19.locate.where(df19.UUIDs != id5,5)
df19.locate = df19.locate.where(df19.UUIDs != id6,6)
df19.locate = df19.locate.where(df19.UUIDs != id7,7)
df19.locate = df19.locate.where(df19.UUIDs != id8,8)
#df19.locate = df19.locate.where(df19.UUIDs != id9,9)
df19.locate = df19.locate.astype(int)

#deviceを作成
df19["device"] = 0
df19.device = df19.device.where(df19.device_name != "rasp1",1)
df19.device = df19.device.where(df19.device_name != "nkmr-2",2)
df19.device = df19.device.where(df19.device_name != "rasp3",3)
df19.device = df19.device.where(df19.device_name != "nkmr-4",3)

#座標を与える
#df19["x"] = 0
#df19["y"] = 0
#df19.x,df17.y = df17.x.where(df17.UUIDs != id1,0),df17.y.where(df17.UUIDs != id1,0)
#df19.x,df17.y = df17.x.where(df17.UUIDs != id2,0),df17.y.where(df17.UUIDs != id2,100)
#df19.x,df17.y = df17.x.where(df17.UUIDs != id3,0),df17.y.where(df17.UUIDs != id3,200)
#df19.x,df17.y = df17.x.where(df17.UUIDs != id4,100),df17.y.where(df17.UUIDs != id4,0)
#df19.x,df17.y = df17.x.where(df17.UUIDs != id5,100),df17.y.where(df17.UUIDs != id5,100)
#df19.x,df17.y = df17.x.where(df17.UUIDs != id6,100),df17.y.where(df17.UUIDs != id6,200)
#df19.x,df17.y = df17.x.where(df17.UUIDs != id7,200),df17.y.where(df17.UUIDs != id7,0)
#df19.x,df17.y = df17.x.where(df17.UUIDs != id8,200),df17.y.where(df17.UUIDs != id8,100)
#df19.x,df17.y = df17.x.where(df17.UUIDs != id9,200),df17.y.where(df17.UUIDs != id9,200)

#発信機ごとに分ける
df19_1 = df19[(df19.UUIDs == id1) | (df19.UUIDs == set_uuid)]
df19_2 = df19[(df19.UUIDs == id2) | (df19.UUIDs == set_uuid)]
df19_3 = df19[(df19.UUIDs == id3) | (df19.UUIDs == set_uuid)]
df19_4 = df19[(df19.UUIDs == id4) | (df19.UUIDs == set_uuid)]
df19_5 = df19[(df19.UUIDs == id5) | (df19.UUIDs == set_uuid)]
df19_6 = df19[(df19.UUIDs == id6) | (df19.UUIDs == set_uuid)]
df19_7 = df19[(df19.UUIDs == id7) | (df19.UUIDs == set_uuid)]
df19_8 = df19[(df19.UUIDs == id8) | (df19.UUIDs == set_uuid)]
#df19_9 = df19[(df19.UUIDs == id9) | (df19.UUIDs == set_uuid)]

df_list = [df19_1,df19_2,df19_3,df19_4,df19_5,df19_6,df19_7,df19_8]

In [4]:
#　オブジェクト型の目的変数を数値情報に変換
import sklearn.preprocessing as sp

# 前処理用のインスタンスを宣言
le = sp.LabelEncoder()

def set_time(data,set_uuid):
    """
    時刻合わせを行う
    第一引数はdataframe
    第２引数は時刻合わせを行うUUID
    """
    # 時刻合わせ用のビーコンを1秒間隔に設定
    set_time_df = data[data.UUIDs == set_uuid].drop_duplicates(['Minor'],keep = "first")
    # 時刻合わせ用ビーコン以外を定義
    df = data[data.UUIDs != set_uuid]
    # データを結合してインデックスでソート
    df = df.append(set_time_df).sort_index(ascending=True)
    # 時刻合わせ用ビーコンを取得した行にtimeカラムを設定
    df["time"] = df["Minor"].where(df["UUIDs"] == set_uuid)
    # 時刻合わせを行い欠損値がある部分は削除
    df = df.fillna(method="ffill").dropna()
    # int型に変換
    df["time"] = df["time"].astype(int)
    # 時刻合わせ用のビーコンデータ以外
    df = df[df["UUIDs"] != set_uuid]
    
    return df

# 前処理を各データフレームに行う
for (i,df) in enumerate(df_list):
    df_list[i] = set_time(df,set_uuid)
    # device_nameを変換
    #df_list[i]["device"] = le.fit_transform(df_list[i][df_list[i].columns[7]]) + 1

In [5]:
# KMeansライブラリをインポート
from sklearn.cluster import KMeans

In [6]:
def make_cluster(df_list,colum,*,N_CLUSTERS=2):
    """
          反射波と入射波を判別するクラスタを作成し描写を行う
          第1引数(地点毎のデータフレームが入ったリスト)
          第２引数(クラスタリングを行いたいカラムのリスト)
          第３引数(クラスタ数)
          データフレーム型でクラスタリング結果を返す
    """

    # 空のデータフレームを用意
    df_cluster = pd.DataFrame()
    concat_list = []
    #座標ごとに
    for df in df_list: 
        #受信機ごとに
        for dev in df.device_name.unique():
            #受信機ごとに分けたdataframe
            df_dev = df[df.device_name == dev].copy()
            # 指定されたカラムを利用してクラスタリング
            pred = KMeans(n_clusters=N_CLUSTERS).fit_predict(np.array(df_dev[colum]))
            # clusterカラムを作成し予測結果を代入
            df_dev.loc[:,("cluster")] = pred
            # RSSI値の平均値を比較して入射波と反射波の判定を行う
            if df_dev[df_dev["cluster"] == 0]["RSSI"].mean() > df_dev[df_dev["cluster"] == 1]["RSSI"].mean():
                pass
            else:
                df_dev.loc[df_dev["cluster"] == 0,"cluster"] = 3
                df_dev.loc[df_dev["cluster"] == 1,"cluster"] = 0
                df_dev.loc[df_dev["cluster"] == 3,"cluster"] = 1
            # 予測結果を格納
            df_cluster = pd.concat([df_cluster,df_dev],ignore_index=True)
        concat_list.append(df_cluster)
        # 空のデータフレームを用意
        df_cluster = pd.DataFrame()

    return concat_list

In [7]:
df_list = make_cluster(df_list,["RSSI","locate"]) 